In [30]:
import pandas as pd
import math
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import re
import gc

import warnings
warnings.filterwarnings("ignore")

In [2]:
scores_cs = pd.read_csv("corpus/cs-en/scores.csv")
scores_de = pd.read_csv("corpus/de-en/scores.csv")
scores_en_fi = pd.read_csv("corpus/en-fi/scores.csv")
scores_en_zh = pd.read_csv("corpus/en-zh/scores.csv")
scores_ru = pd.read_csv("corpus/ru-en/scores.csv")
scores_zh = pd.read_csv("corpus/zh-en/scores.csv")

In [3]:
# DEPRECATED - HERE TO SHOW HOW CONTRACTIONS LOOK LIKE

# contractions = { 
# "ain't": "am not / are not / is not / has not / have not",
# "aren't": "are not / am not",
# "can't": "cannot",
# "can't've": "cannot have",
# "'cause": "because",
# "could've": "could have",
# "couldn't": "could not",
# "couldn't've": "could not have",
# "didn't": "did not",
# "doesn't": "does not",
# "don't": "do not",
# "hadn't": "had not",
# "hadn't've": "had not have",
# "hasn't": "has not",
# "haven't": "have not",
# "he'd": "he had / he would",
# "he'd've": "he would have",
# "he'll": "he shall / he will",
# "he'll've": "he shall have / he will have",
# "he's": "he has / he is",
# "how'd": "how did",
# "how'd'y": "how do you",
# "how'll": "how will",
# "how's": "how has / how is / how does",
# "I'd": "I had / I would",
# "I'd've": "I would have",
# "I'll": "I shall / I will",
# "I'll've": "I shall have / I will have",
# "I'm": "I am",
# "I've": "I have",
# "isn't": "is not",
# "it'd": "it had / it would",
# "it'd've": "it would have",
# "it'll": "it shall / it will",
# "it'll've": "it shall have / it will have",
# "it's": "it has / it is",
# "let's": "let us",
# "ma'am": "madam",
# "mayn't": "may not",
# "might've": "might have",
# "mightn't": "might not",
# "mightn't've": "might not have",
# "must've": "must have",
# "mustn't": "must not",
# "mustn't've": "must not have",
# "needn't": "need not",
# "needn't've": "need not have",
# "o'clock": "of the clock",
# "oughtn't": "ought not",
# "oughtn't've": "ought not have",
# "shan't": "shall not",
# "sha'n't": "shall not",
# "shan't've": "shall not have",
# "she'd": "she had / she would",
# "she'd've": "she would have",
# "she'll": "she shall / she will",
# "she'll've": "she shall have / she will have",
# "she's": "she has / she is",
# "should've": "should have",
# "shouldn't": "should not",
# "shouldn't've": "should not have",
# "so've": "so have",
# "so's": "so as / so is",
# "that'd": "that would / that had",
# "that'd've": "that would have",
# "that's": "that has / that is",
# "there'd": "there had / there would",
# "there'd've": "there would have",
# "there's": "there has / there is",
# "they'd": "they had / they would",
# "they'd've": "they would have",
# "they'll": "they shall / they will",
# "they'll've": "they shall have / they will have",
# "they're": "they are",
# "they've": "they have",
# "to've": "to have",
# "wasn't": "was not",
# "we'd": "we had / we would",
# "we'd've": "we would have",
# "we'll": "we will",
# "we'll've": "we will have",
# "we're": "we are",
# "we've": "we have",
# "weren't": "were not",
# "what'll": "what shall / what will",
# "what'll've": "what shall have / what will have",
# "what're": "what are",
# "what's": "what has / what is",
# "what've": "what have",
# "when's": "when has / when is",
# "when've": "when have",
# "where'd": "where did",
# "where's": "where has / where is",
# "where've": "where have",
# "who'll": "who shall / who will",
# "who'll've": "who shall have / who will have",
# "who's": "who has / who is",
# "who've": "who have",
# "why's": "why has / why is",
# "why've": "why have",
# "will've": "will have",
# "won't": "will not",
# "won't've": "will not have",
# "would've": "would have",
# "wouldn't": "would not",
# "wouldn't've": "would not have",
# "y'all": "you all",
# "y'all'd": "you all would",
# "y'all'd've": "you all would have",
# "y'all're": "you all are",
# "y'all've": "you all have",
# "you'd": "you had / you would",
# "you'd've": "you would have",
# "you'll": "you shall / you will",
# "you'll've": "you shall have / you will have",
# "you're": "you are",
# "you've": "you have"
# }

# contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
# def expand_contractions(s, contractions = contractions):
#     def replace(match):
#         return contractions[match.group(0)]
#     return contractions_re.sub(replace, s)

# expand_contractions("ain't stop")


In [32]:
# Same as previous cell but pycontractes choose a proper form
# instead ambiguous "you'd" -> "you had / you would"
# using contractions lib pycontractions
from pycontractions import Contractions

cont = Contractions('GoogleNews-vectors-negative300.bin')

# optional, prevents loading on first expand_texts call
cont.load_models()

In [88]:
a = scores_de.translation
a[6]

"It's now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work."

In [87]:
b = pd.Series(list(cont.expand_texts(a, precise=True)))

In [84]:
b[6]

'it is now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.'

In [89]:
# As contraction extractions works really slow
# it was done outside preprocessing function
references_cs = pd.Series(list(cont.expand_texts(scores_cs['reference'], precise=True)))
references_de = pd.Series(list(cont.expand_texts(scores_de['reference'], precise=True)))
references_ru = pd.Series(list(cont.expand_texts(scores_ru['reference'], precise=True)))
references_zh = pd.Series(list(cont.expand_texts(scores_zh['reference'], precise=True)))

In [91]:
# Same for transaltions
translation_cs = pd.Series(list(cont.expand_texts(scores_cs['translation'], precise=True)))
translation_de = pd.Series(list(cont.expand_texts(scores_de['translation'], precise=True)))
translation_ru = pd.Series(list(cont.expand_texts(scores_ru['translation'], precise=True)))
translation_zh = pd.Series(list(cont.expand_texts(scores_zh['translation'], precise=True)))

In [96]:
# Check how expanding contractions worked
print(scores_de['translation'][6], '   ---->  \n',translation_de[6])

It's now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.    ---->  
 it is now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.


In [ ]:
# release references and 'garbage collect' to free memory
import gc
del cont
gc.collect()

In [99]:
def preprocessing(series, pontuation = False):
    processed_corpus = []
    #stop_words = set(stopwords.words())
    for i in tqdm(range(len(series))):
        text = series[i]
        
        #remove tags
        text = BeautifulSoup(text).get_text()
        
        #Remove punctuations
        if pontuation==True:
            text = re.sub(r'[\s+\.\!\/_,$%^*(+\"\']+|[+——！，？、~@#￥%……&*（）:：；《）《》“”()»〔〕-]+', ' ', text)
        else:
            pass

        #Convert to lowercase
        text = text.lower()
        
        # Convert to list from string
        text = text.split()

        #Lemmatisation
        lem = WordNetLemmatizer()
        #text = [lem.lemmatize(word) for word in text if not word in stop_words] 
        text = " ".join(text)
        processed_corpus.append(text)
    return processed_corpus

In [100]:
references_cs = preprocessing(references_cs)
references_de = preprocessing(references_de)
references_en_fi = preprocessing(scores_en_fi['reference'])
references_en_zh = preprocessing(scores_en_zh['reference'])
references_ru = preprocessing(references_ru)
references_zh = preprocessing(references_zh)

<ipython-input-99-c106e6b4224f>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(series))):


C:\Users\murmu\anaconda3\lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [101]:
translation_cs = preprocessing(translation_cs)
translation_de = preprocessing(translation_de)
translation_en_fi = preprocessing(scores_en_fi['translation'])
translation_en_zh = preprocessing(scores_en_zh['translation'])
translation_ru = preprocessing(translation_ru)
translation_zh = preprocessing(translation_zh)

<ipython-input-99-c106e6b4224f>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(series))):


In [114]:
# Import libraries
import nltk.translate.chrf_score as chrf
from rouge import Rouge
import nltk
import nltk.translate.gleu_score as gleu

import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')
import nltk.translate.meteor_score as meteor
import nltk.translate.nist_score as nist
import nltk.translate.bleu_score as bleu

In [ ]:
# METRICS from text_mining1

In [107]:
def chrf_metric(translation, references):
    chrf_metric = []
    for i in range(len(translation)):
        row = chrf.sentence_chrf([translation[i]], references[i])
        chrf_metric.append(row)
    return chrf_metric

chf_cs = chrf_metric(translation_cs, references_cs)
scores_cs['chf'] = chf_cs
chf_de = chrf_metric(translation_de, references_de)
scores_de['chf'] = chf_de
chf_en_fi = chrf_metric(translation_en_fi, references_en_fi)
scores_en_fi['chf'] = chf_en_fi
chf_en_zh = chrf_metric(translation_en_zh, references_en_zh)
scores_en_zh['chf'] = chf_en_zh
chf_ru = chrf_metric(translation_ru, references_ru)
scores_ru['chf'] = chf_ru
chf_zh = chrf_metric(translation_zh, references_zh)
scores_zh['chf'] = chf_zh

In [109]:
def gleu_metric(translation, references):
    gleu_metric = []
    for i in range(len(translation)):
        row = gleu.sentence_gleu([str(references[i]).split()], str(translation[i]).split(), min_len=1, max_len=2)
        gleu_metric.append(row)
    return gleu_metric

In [110]:
gleu_cs = gleu_metric(translation_cs, references_cs)
scores_cs['gleu'] = gleu_cs

gleu_de = gleu_metric(translation_de, references_de)
scores_de['gleu'] = gleu_de

gleu_en_fi = gleu_metric(translation_en_fi, references_en_fi)
scores_en_fi['gleu'] = gleu_en_fi

gleu_en_zh = gleu_metric(translation_en_zh, references_en_zh)
scores_en_zh['gleu'] = gleu_en_zh

gleu_ru = gleu_metric(translation_ru, references_ru)
scores_ru['gleu'] = gleu_ru

gleu_zh = gleu_metric(translation_zh, references_zh)
scores_zh['gleu'] = gleu_zh

In [111]:
def meteor_metric(translation, references):
    meteor_metric = []
    for i in range(len(translation)):
        row = (meteor.meteor_score([translation[i]], references[i]))
        meteor_metric.append(row)
    return meteor_metric

In [112]:
meteor_cs = meteor_metric(translation_cs, references_cs)
scores_cs['meteor'] = meteor_cs

meteor_de = meteor_metric(translation_de, references_de)
scores_de['meteor'] = meteor_de

meteor_en_fi = meteor_metric(translation_en_fi, references_en_fi)
scores_en_fi['meteor'] = meteor_en_fi

meteor_en_zh = meteor_metric(translation_en_zh, references_en_zh)
scores_en_zh['meteor'] = meteor_en_zh

meteor_ru = meteor_metric(translation_ru, references_ru)
scores_ru['meteor'] = meteor_ru

meteor_zh = meteor_metric(translation_zh, references_zh)
scores_zh['meteor'] = meteor_zh

In [115]:
def bleu_metric(translation, references):
    bleu_metric = []
    for i in range(len(translation)):
        row = (bleu.sentence_bleu([translation[i]], references[i]))
        bleu_metric.append(row)
    return bleu_metric

bleu_cs = bleu_metric(translation_cs, references_cs)
scores_cs['bleu'] = bleu_cs

bleu_de = bleu_metric(translation_de, references_de)
scores_de['bleu'] = bleu_de

bleu_en_fi = bleu_metric(translation_en_fi, references_en_fi)
scores_en_fi['bleu'] = bleu_en_fi

bleu_en_zh = bleu_metric(translation_en_zh, references_en_zh)
scores_en_zh['bleu'] = bleu_en_zh

bleu_ru = bleu_metric(translation_ru, references_ru)
scores_ru['bleu'] = bleu_ru

bleu_zh = bleu_metric(translation_zh, references_zh)
scores_zh['bleu'] = bleu_zh

C:\Users\murmu\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\murmu\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\murmu\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin

In [116]:

def nist_metric(translation, references):
    nist_metric = []
    for i in range(len(translation)):
        row = (nist.sentence_nist([translation[i]], references[i],n=1))
        nist_metric.append(row)
    return nist_metric

In [117]:
nist_cs = nist_metric(translation_cs, references_cs)
scores_cs['nist'] = nist_cs

nist_de = nist_metric(translation_de, references_de)
scores_de['nist'] = nist_de

nist_en_fi = nist_metric(translation_en_fi, references_en_fi)
scores_en_fi['nist'] = nist_en_fi

nist_en_zh = nist_metric(translation_en_zh, references_en_zh)
scores_en_zh['nist'] = nist_en_zh

nist_ru = nist_metric(translation_ru, references_ru)
scores_ru['nist'] = nist_ru

nist_zh = nist_metric(translation_zh, references_zh)
scores_zh['nist'] = nist_zh

In [121]:
def corr(scores):
    chf = scores['z-score'].corr(scores['chf'])
    gleu = scores['z-score'].corr(scores['gleu'])
    meteor = scores['z-score'].corr(scores['meteor'])
    bleu = scores['z-score'].corr(scores['bleu'])
    nist = scores['z-score'].corr(scores['nist'])
    #ribes = scores['z-score'].corr(scores['ribes'])
    corr = [chf, gleu, meteor, bleu, nist]
    return corr

In [122]:
corr_cs = corr(scores_cs)
corr_de = corr(scores_de)
corr_en_fi = corr(scores_en_fi)
corr_en_zh = corr(scores_en_zh)
corr_ru = corr(scores_ru)
corr_zh = corr(scores_zh)

In [123]:
columns = ['chf', 'gleu', 'meteor', 'bleu', 'nist']
data = []
index = ['scores_cs', 'scores_de', 'scores_en_fi', 'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)
correlation.iloc[0] = corr_cs
correlation.iloc[1] = corr_de
correlation.iloc[2] = corr_en_fi
correlation.iloc[3] = corr_en_zh
correlation.iloc[4] = corr_ru
correlation.iloc[5] = corr_zh

In [124]:
correlation

,chf,gleu,meteor,bleu,nist
scores_cs,0.462233,0.427893,0.440004,0.468779,0.334652
scores_de,0.341149,0.310098,0.30805,0.346757,0.227363
scores_en_fi,0.611567,0.494636,0.491475,0.619928,0.423513
scores_en_zh,0.435736,0.0296857,0.0231177,0.433811,0.444757
scores_ru,0.361417,0.333536,0.336745,0.367581,0.26207
scores_zh,0.341204,0.317924,0.326358,0.351904,0.244659
